In [9]:
#import necessary libraries
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics

In [29]:
#import dataset
df = pd.read_csv("energydata_complete.csv")
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [4]:
df.shape

(19735, 29)

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19735 entries, 0 to 19734
Data columns (total 29 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         19735 non-null  object 
 1   Appliances   19735 non-null  int64  
 2   lights       19735 non-null  int64  
 3   T1           19735 non-null  float64
 4   RH_1         19735 non-null  float64
 5   T2           19735 non-null  float64
 6   RH_2         19735 non-null  float64
 7   T3           19735 non-null  float64
 8   RH_3         19735 non-null  float64
 9   T4           19735 non-null  float64
 10  RH_4         19735 non-null  float64
 11  T5           19735 non-null  float64
 12  RH_5         19735 non-null  float64
 13  T6           19735 non-null  float64
 14  RH_6         19735 non-null  float64
 15  T7           19735 non-null  float64
 16  RH_7         19735 non-null  float64
 17  T8           19735 non-null  float64
 18  RH_8         19735 non-null  float64
 19  T9  

## Simple Linear Regression

In [22]:
X = df["T2"].values
y = df["T6"].values
X = X.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [23]:
model = LinearRegression()
model.fit(X, y)
preds = model.predict(X_test)

In [24]:
# regression performance
r2_score = metrics.r2_score(y_test, preds)
round(r2_score, 3)

0.643

## Pre-processing

Normalizing the dataset to a commom scale using MinMaxScaler before building model.

In [31]:
df = df.drop(columns=["date", "lights"], axis=1)
scaler = MinMaxScaler()
df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

## Multiple Linear Regression

### Linear Regression

In [33]:
X = df.drop("Appliances", axis = 1)
y = df["Appliances"]

#splitting dataset to train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

lin = LinearRegression()
lin.fit(X_train, y_train)
pred = lin.predict(X_test)

#### Regression performance

In [34]:
metrics.mean_absolute_error(y_test, pred)

0.05013362397742955

In [40]:
rss = np.sum(np.square(y_test - pred))
round(rss,2)

45.35

In [36]:
rmse = np.sqrt(metrics.mean_squared_error(y_test, pred))
round(rmse, 3)

0.088

In [38]:
round(metrics.r2_score(y_test, pred), 2)

0.15

#### Ridge

In [58]:
ridge_reg = Ridge(alpha=0.4)
ridge_reg.fit(X_train, y_train)
round(np.sqrt(metrics.mean_squared_error(y_test, ridge_reg.predict(X_test))), 3)

0.088

#### Lasso

In [59]:
lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(X_train,y_train)
lasso_reg.predict(X_test)

array([0.07370267, 0.08143458, 0.07716072, ..., 0.07792848, 0.09034412,
       0.08359255])

In [60]:
round(np.sqrt(metrics.mean_squared_error(y_test, lasso_reg.predict(X_test))), 3)

0.094

### Features selection

In [39]:
def get_weights_df(model, feat, col_name):
    weights = pd.Series(model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ['Features', col_name]
    weights_df[col_name].round(3)
    return weights_df

In [61]:
lin_weights = get_weights_df(lin, X_train, 'Linear_Model_Weight')
lasso_weights_df = get_weights_df(lasso_reg, X_train, 'Lasso_weight')
ridge_weights_df = get_weights_df(ridge_reg, X_train, 'Ridge_Weight')

final_weights = pd.merge(lin_weights, ridge_weights_df, on='Features')
final_weights =pd.merge(final_weights, lasso_weights_df, on='Features')

final_weights

,Features,Linear_Model_Weight,Ridge_Weight,Lasso_weight
0,RH_2,-0.456698,-0.411071,-0.000000
1,T_out,-0.321860,-0.262172,0.000000
2,T2,-0.236178,-0.201397,0.000000
3,T9,-0.189941,-0.188916,-0.000000
4,RH_8,-0.157595,-0.156830,-0.000110
5,RH_out,-0.077671,-0.054724,-0.049557
6,RH_7,-0.044614,-0.045977,-0.000000
7,RH_9,-0.039800,-0.041367,-0.000000
8,T5,-0.015657,-0.019853,-0.000000
9,T1,-0.003281,-0.018406,0.000000


In [63]:
final_weights.max()

Features                    rv2
Linear_Model_Weight    0.553547
Ridge_Weight           0.519525
Lasso_weight            0.01788
dtype: object

In [64]:
final_weights.min()

Features               Press_mm_hg
Linear_Model_Weight      -0.456698
Ridge_Weight             -0.411071
Lasso_weight             -0.049557
dtype: object